In [ ]:
%load_ext autoreload
%autoreload 2
import pyceres
import pycolmap
import numpy as np
from hloc.utils import viz_3d
from copy import deepcopy
import cv2

## Setup the toy example

In [ ]:
def sample_rotation(max_=np.pi*2):
    aa = np.random.randn(3)
    aa *= np.random.rand()*max_ / np.linalg.norm(aa)
    R = cv2.Rodrigues(aa)[0]
    qvec = pycolmap.rotmat_to_qvec(R)
    return qvec

def invert(q, t):
    return (pycolmap.invert_qvec(q), -pycolmap.qvec_to_rotmat(q).T@t)

def error(qt1, qt2):
    q, t = pycolmap.relative_pose(*qt1, *qt2)
    return (np.linalg.norm(cv2.Rodrigues(pycolmap.qvec_to_rotmat(q))[0]), np.linalg.norm(t))

num = 20
qt_w_i = [(sample_rotation(), np.random.rand(3)*10) for _ in range(num)]
qt_i_w = [invert(q, t) for q, t in qt_w_i]

qt_i_j = [pycolmap.relative_pose(*qt_i_w[(i+1)%num], *qt_i_w[i]) for i in range(num)]

qt_i_w_init = [(pycolmap.rotmat_to_qvec(pycolmap.qvec_to_rotmat(q)
                                        @ pycolmap.qvec_to_rotmat(sample_rotation(np.pi/5))),
                t+np.random.randn(3)) for q, t in qt_i_w]
qt_i_w_init[0] = qt_i_w[0]

## PGO with relative poses

In [ ]:
qt_i_w_opt = deepcopy(qt_i_w_init)

prob = pyceres.Problem()
loss = pyceres.TrivialLoss()
costs = []
for i in range(num):
    cost = pyceres.factors.PoseGraphRelativeCost(*invert(*qt_i_j[i]), np.eye(6))
    costs.append(cost)
    prob.add_residual_block(cost, loss, [*qt_i_w_opt[i], *qt_i_w_opt[(i+1)%num]])
    prob.set_parameterization(qt_i_w_opt[i][0], pyceres.QuaternionParameterization())
prob.set_parameter_block_constant(qt_i_w_opt[0][0])
prob.set_parameter_block_constant(qt_i_w_opt[0][1])

options = pyceres.SolverOptions()
options.linear_solver_type = pyceres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = False
options.num_threads = -1
summary = pyceres.SolverSummary()
pyceres.solve(options, prob, summary)
print(summary.BriefReport())

err_init = np.array([error(qt_i_w[i], qt_i_w_init[i]) for i in range(num)])
err_opt = np.array([error(qt_i_w[i], qt_i_w_opt[i]) for i in range(num)])
print(np.mean(err_opt, 0))

In [ ]:
qt_i_j_init = [pycolmap.relative_pose(*qt_i_w_init[(i+1)%num], *qt_i_w_init[i]) for i in range(num)]
for i in range(num):
    error_rel_init = error(invert(*qt_i_j[i]), invert(*qt_i_j_init[i]))  # qt_j_init_j
    res = costs[i].evaluate(*qt_i_w_init[i], *qt_i_w_init[(i+1)%num])[0]
    error_rel_init_ceres = (np.linalg.norm(res[:3]), np.linalg.norm(res[3:]))
    assert np.allclose(error_rel_init, error_rel_init_ceres)

# PGO with absolute pose

In [ ]:
qt_i_w_opt = deepcopy(qt_i_w_init)

prob = pyceres.Problem()
loss = pyceres.TrivialLoss()
costs = []
for i in range(num):
    cost = pyceres.factors.PoseGraphAbsoluteCost(*qt_i_w[i], np.eye(6))
    costs.append(cost)
    prob.add_residual_block(cost, loss, [*qt_i_w_opt[i]])
    prob.set_parameterization(qt_i_w_opt[i][0], pyceres.QuaternionParameterization())

options = pyceres.SolverOptions()
# options.linear_solver_type = pyceres.LinearSolverType.DENSE_QR
options.linear_solver_type = pyceres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = False
options.num_threads = -1
summary = pyceres.SolverSummary()
pyceres.solve(options, prob, summary)
print(summary.BriefReport())

err_init = np.array([error(qt_i_w[i], qt_i_w_init[i]) for i in range(num)])
err_opt = np.array([error(qt_i_w[i], qt_i_w_opt[i]) for i in range(num)])
print(np.mean(err_opt, 0))